# retrieve run data from mongo

In [72]:
from pymongo import MongoClient
import polars as pl

client = MongoClient("localhost", 27017)
db = client["trader"]
collection = db["message"]
documents = list(collection.find({}))  # Retrieve all documents
run_data = pl.DataFrame(documents)
run_data

_id,trading_session_id,content,timestamp
object,str,struct[10],datetime[μs]
66562bf3d07ba87713a3c364,"""13a876ca-9eb8-…","{""Market is open"",""update"",{[],[]},[],[],null,null,null,{null,null,null,null},null}",2024-05-28 20:09:39.205
66562bf3d07ba87713a3c365,"""13a876ca-9eb8-…","{null,""update"",{[],[{2000.0,1.0}]},[{b""g\xd4\x20i&\x89I<\xa9\xfb\xb0\xdb\x1151\xad"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2000.0,2024-05-28 19:09:39.271}],[],null,null,null,{1,2000.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""},""book is updated""}",2024-05-28 20:09:39.322
66562bf3d07ba87713a3c366,"""13a876ca-9eb8-…","{null,""update"",{[],[{2000.0,1.0}, {2001.0,1.0}]},[{b""g\xd4\x20i&\x89I<\xa9\xfb\xb0\xdb\x1151\xad"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2000.0,2024-05-28 19:09:39.271}, {b""\x88F\x97E_\x91D\xff\xb9\xef\x02}a\xd2}\x8d"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2001.0,2024-05-28 19:09:39.323}],[],null,null,null,{1,2001.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""},""book is updated""}",2024-05-28 20:09:39.373
66562bf5d07ba87713a3c367,"""13a876ca-9eb8-…","{null,""update"",{[],[{2000.0,1.0}, {2001.0,1.0}, {2002.0,1.0}]},[{b""g\xd4\x20i&\x89I<\xa9\xfb\xb0\xdb\x1151\xad"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2000.0,2024-05-28 19:09:39.271}, {b""\x88F\x97E_\x91D\xff\xb9\xef\x02}a\xd2}\x8d"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2001.0,2024-05-28 19:09:39.323}, {b""K\xa6\x99\xd0\xfb\xbaM\xc0\xac\x9bd\x97\xaf\x00Q\x0b"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2002.0,2024-05-28 19:09:41.576}],[],null,null,null,{1,2002.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""},""book is updated""}",2024-05-28 20:09:41.640
66562bf6d07ba87713a3c368,"""13a876ca-9eb8-…","{null,""update"",{[],[{2000.0,1.0}, {2001.0,1.0}, … {2003.0,1.0}]},[{b""g\xd4\x20i&\x89I<\xa9\xfb\xb0\xdb\x1151\xad"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2000.0,2024-05-28 19:09:39.271}, {b""\x88F\x97E_\x91D\xff\xb9\xef\x02}a\xd2}\x8d"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2001.0,2024-05-28 19:09:39.323}, … {b""\x17\xac\x0bS\xacCLT\xbe\x8c\xd4\x18\x19\xca\xf0\x08"",""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d"",-1,1.0,2003.0,2024-05-28 19:09:42.903}],[],null,null,null,{1,2003.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""},""book is updated""}",2024-05-28 20:09:42.972
…,…,…,…
66562c2fd07ba87713a3c374,"""13a876ca-9eb8-…","{null,""stop_trading"",{[],[]},[],[{""825b8de0-4840-44bd-bb12-133636d8156c"",""13a876ca-9eb8-4fc5-af33-1c502de08e6a"",""65720809-f496-4b75-b6a2-717bae647595"",""67d42069-2689-493c-a9fb-b0db113531ad"",2024-05-28 20:10:39.340,990.0}, {""0fdc469a-e21f-4a9c-aa88-12e05b41a6ce"",""13a876ca-9eb8-4fc5-af33-1c502de08e6a"",""1b12fd39-ad0d-4740-b299-8bd6b85ad549"",""88469745-5f91-44ff-b9ef-027d61d27d8d"",2024-05-28 20:10:39.345,990.0}, … {""42407312-4385-4997-919b-a8ca726d8c46"",""13a876ca-9eb8-4fc5-af33-1c502de08e6a"",""29ac6b5b-04f5-4ea7-8c1e-1b933653c8ba"",""3d4b45f9-92e5-4d7b-9529-f8fd0da4da28"",2024-05-28 20:10:39.371,1010.0}],null,null,1010.0,{null,null,null,null},null}",2024-05-28 20:10:39.789
66562e09204af8443336711f,"""ae080315-f67e-…","{""Market is open"",""update"",{[],[]},[],[],null,null,null,{null,null,null,null},null}",2024-05-28 20:18:33.098
66562e0a204af84433367120,"""ae080315-f67e-…","{null,""update"",{[{2000.0,1.0}],[]},[{b""k\x84\x04\x93\xa3QNw\x93\x96;q\x14\xca\xd62"",""NOISE_557ff4e7-b944-4d1a-9476-9edaff99f8f1"",1,1.0,2000.0,2024-05-28 19:18:34.223}],[],null,null,null,{1,2000.0,1,""NOISE_557ff4e7-b944-4d1a-9476-9edaff99f8f1""},""book is updated""}",2024-05-28 20:18:34.280


In [54]:
run_data['content'][1]

{'content': None,
 'type': 'update',
 'order_book': {'bids': [], 'asks': [{'x': 2000.0, 'y': 1.0}]},
 'active_orders': [{'id': b'g\xd4 i&\x89I<\xa9\xfb\xb0\xdb\x1151\xad',
   'trader_id': 'NOISE_32e40843-ae17-4397-ab28-e1259b8a848d',
   'order_type': -1,
   'amount': 1.0,
   'price': 2000.0,
   'timestamp': datetime.datetime(2024, 5, 28, 19, 9, 39, 271000)}],
 'history': [],
 'spread': None,
 'midpoint': None,
 'transaction_price': None,
 'incoming_message': {'amount': 1,
  'price': 2000.0,
  'order_type': -1,
  'trader_id': 'NOISE_32e40843-ae17-4397-ab28-e1259b8a848d'},
 'text': 'book is updated'}

'13a876ca-9eb8-4fc5-af33-1c502de08e6a'

In [47]:
from analysis.utilities import delete_all_tables

delete_all_tables()

DuckDB tables deleted successfully.
MongoDB collection deleted successfully.


In [74]:
import polars as pl

# Filter rows where trading_session_id is '71062ff9-b112-4253-bccb-fc054640d17a'
filtered_data = run_data.filter(pl.col("trading_session_id") == run_data['trading_session_id'][-1])

# Preprocess data in Python
preprocessed_data = []
for row in filtered_data.to_dicts():
    content = row['content']
    order_book = content.get('order_book', None)
    incoming = content.get('incoming_message', None)
    timestamp = row['timestamp']  # Correctly access timestamp from row
    
    preprocessed_data.append({
        'order_book': order_book,
        'incoming': incoming,
        'timestamp': timestamp  # Include timestamp in the dictionary
    })

# Create a new DataFrame from preprocessed data
new_order_books = pl.DataFrame(preprocessed_data)

# Sort by 'timestamp', handling None values by sorting them last
new_order_books = new_order_books.sort("timestamp", nulls_last=True)

# Display the DataFrame
print(new_order_books)

shape: (4, 3)
┌─────────────────────┬───────────────────────────────────┬─────────────────────────┐
│ order_book          ┆ incoming                          ┆ timestamp               │
│ ---                 ┆ ---                               ┆ ---                     │
│ struct[2]           ┆ struct[4]                         ┆ datetime[μs]            │
╞═════════════════════╪═══════════════════════════════════╪═════════════════════════╡
│ {[],[]}             ┆ {null,null,null,null}             ┆ 2024-05-28 20:18:33.098 │
│ {[{2000.0,1.0}],[]} ┆ {1,2000.0,1,"NOISE_557ff4e7-b944… ┆ 2024-05-28 20:18:34.280 │
│ {[{2001.0,1.0}],[]} ┆ {1,2000.0,-1,"NOISE_557ff4e7-b94… ┆ 2024-05-28 20:18:37.407 │
│ {[{2001.0,1.0}],[]} ┆ {1,2001.0,1,"NOISE_557ff4e7-b944… ┆ 2024-05-28 20:18:37.467 │
└─────────────────────┴───────────────────────────────────┴─────────────────────────┘


In [71]:
from main_platform.utils import convert_to_book_format_new
dt = convert_to_book_format_new(new_order_books['order_book'][2])

dt[5]

1.0

order_book,incoming,timestamp
struct[2],struct[4],datetime[μs]
"{[],[]}","{null,null,null,null}",2024-05-28 20:09:39.205
"{[],[{2000.0,1.0}]}","{1,2000.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:39.322
"{[],[{2000.0,1.0}, {2001.0,1.0}]}","{1,2001.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:39.373
"{[],[{2000.0,1.0}, {2001.0,1.0}, {2002.0,1.0}]}","{1,2002.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:41.640
"{[],[{2000.0,1.0}, {2001.0,1.0}, … {2003.0,1.0}]}","{1,2003.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:42.972
"{[],[{2000.0,1.0}, {2001.0,1.0}, … {2004.0,1.0}]}","{1,2004.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:44.144
"{[],[{2000.0,1.0}, {2001.0,1.0}, … {2005.0,1.0}]}","{1,2005.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:44.200
"{[],[{2000.0,1.0}, {2001.0,1.0}, … {2006.0,1.0}]}","{1,2006.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:44.263
"{[],[{2000.0,1.0}, {2001.0,1.0}, … {2007.0,1.0}]}","{1,2007.0,-1,""NOISE_32e40843-ae17-4397-ab28-e1259b8a848d""}",2024-05-28 20:09:46.463


In [21]:
timestamp

In [4]:
run_data['content'][1]['order_book']

{'bids': [], 'asks': [{'x': 2008.0, 'y': 1.0}]}